In [27]:
import pandas as pd
from omop_etl.datastore import DataStore
from omop_etl.stage import STAGE

from pydma.utils import find

In [15]:
omop = DataStore('../config.yml')

In [16]:
stage_t = omop.list_tables(in_schema=['stage'])
stage_t['Count'] = stage_t.Table.apply(lambda t: omop.row_count(t, schema='stage'))

In [17]:
stage_t_dict = stage_t[['Table','Count']].to_dict()
# stage_t_dict

In [18]:
stage_t_dict['Count'][7]

73484

In [19]:
def find_key(input_dict, value):
    return next((k for k, v in input_dict.items() if v == value), None)

In [20]:
loaded = omop.config_param['load']
result = []
for t in loaded.keys():
    if t not in ['provider','care_site','location']:
        if loaded[t]:
            for part in loaded[t]:
                stg_name = STAGE[t][part]
                idx = find_key(stage_t_dict['Table'], stg_name)
                count = stage_t_dict['Count'][idx]
                result.append([t, part, count])
        else:
            stg_name = STAGE[t]
            idx = find_key(stage_t_dict['Table'], stg_name)
            count = stage_t_dict['Count'][idx]
            result.append([t, None, count])

table_counts = pd.DataFrame(result, columns=['Table', 'Part', 'Count'])
table_counts.head()

,Table,Part,Count
0,person,None,242390
1,death,None,5579
2,condition_occurrence,None,25348522
3,procedure_occurrence,icd,174452
4,procedure_occurrence,cpt,11027439


In [21]:
count_diff = table_counts.groupby('Table').sum().reset_index()
count_diff['RC_Load'] = count_diff.Table.apply(lambda t: omop.row_count(t))
count_diff['RC_DeId'] = count_diff.Table.apply(lambda t: omop.row_count(t, schema='deid'))
count_diff

,Table,Count,RC_Load,RC_DeId
0,condition_occurrence,25348522,20867090,20867090
1,death,5579,5579,5579
2,drug_exposure,34347650,34173541,34173541
3,measurement,273904063,275651510,275651510
4,observation,33763735,40503251,40503251
5,person,242390,242388,242388
6,procedure_occurrence,11201891,4790264,4790264
7,visit_occurrence,9704407,9669131,9669131


In [32]:
t_sbs = stage_t[find(['MEASUREMENT','DRUG','CONDITION','PROCEDURE','OBSERVATION'], stage_t.Table)].Table.tolist()

['MEASUREMENT_BP',
 'OBSERVATION_Zipcode',
 'OBSERVATION_Smoking',
 'OBSERVATION_Payer',
 'OBSERVATION_Vent',
 'OBSERVATION_LDA',
 'OBSERVATION_ICU',
 'MEASUREMENT_Weight',
 'MEASUREMENT_Temp',
 'MEASUREMENT_SOFA',
 'MEASUREMENT_Rothman',
 'MEASUREMENT_Res_Vent',
 'MEASUREMENT_Res_Tidal',
 'MEASUREMENT_Res_SPO2',
 'MEASUREMENT_Res_RESP',
 'MEASUREMENT_Res_PIP',
 'MEASUREMENT_Res_PEEP',
 'MEASUREMENT_Res_O2',
 'MEASUREMENT_Res_GCS',
 'MEASUREMENT_Res_FIO2',
 'MEASUREMENT_Res_ETCO2',
 'MEASUREMENT_Res_Device',
 'MEASUREMENT_QTCB',
 'MEASUREMENT_PainScale',
 'MEASUREMENT_LAB',
 'MEASUREMENT_Height',
 'MEASUREMENT_HeartRate',
 'DRUG_ADMIN',
 'DRUG_ORDER',
 'PROCEDURE_CPT',
 'PROCEDURE_ICD',
 'CONDITION']